In [1]:
import os
import pandas as pd
from sklearn import preprocessing
import numpy as np
import math
df = pd.read_csv('C:\\gab\\train.csv',
                   sep=',', 
                   header=None, 
                   names=['article', 'label'],
                   encoding='ISO-8859-1')
df1 = pd.read_csv('C:\\gab\\test.csv',
                   sep=',', 
                   header=None, 
                   names=['article', 'label'],
                   encoding='ISO-8859-1')
df2 = pd.read_csv('C:\\gab\\dev.csv',
                   sep=',', 
                   header=None, 
                   names=['article', 'label'],
                   encoding='ISO-8859-1')
df.drop_duplicates(keep='first', inplace=True)   
print(df.dropna(subset=['article', 'label']))
print(df1.dropna(subset=['article', 'label']))
print(df2.dropna(subset=['article', 'label']))

                                                 article  label
0      What's wrong with that? It's History you know....      0
1      Switzerland, I'm guessing ? if so, an outlier ...      0
2      What are they selling ? Are? they trying to? m...      0
3      http://bit.do/b7ndm: Dr. Paul Powers is back i...      0
4      @support? the prompt for browser notifications...      0
...                                                  ...    ...
19088  Now we just need to wall in Israel and it's pa...      0
19089  Gotta have some pretty strong legs to make tha...      0
19090  Rudy Giuliani Is Putting Together a ¡®Counter-...      0
19091                                   Save #FreeSpeech      0
19092  A #British #comics publisher doesn't want whit...      1

[19037 rows x 2 columns]
                                                article  label
0     Please go to Dane's website at www.geoengineer...      0
1     Anybody else sick of online product launch vid...      1
2     Hold on #Ga

In [2]:
import collections
c = collections.Counter(df['label'])
c1 = collections.Counter(df1['label'])
c2 = collections.Counter(df2['label'])
print(c)
print(c1)
print(c2)

Counter({0: 16843, 1: 2194})
Counter({0: 3658, 1: 475})
Counter({0: 3577, 1: 475})


In [3]:
print(df.shape)
df.head() 
df['label'].isnull().value_counts
print(df1.shape)
df1.head() 
df1['label'].isnull().value_counts
print(df2.shape)
df2.head() 
df2['label'].isnull().value_counts

(19037, 2)
(4133, 2)
(4052, 2)


<bound method IndexOpsMixin.value_counts of 0       False
1       False
2       False
3       False
4       False
        ...  
4047    False
4048    False
4049    False
4050    False
4051    False
Name: label, Length: 4052, dtype: bool>

In [4]:
import string
import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
type(df)
stop = stopwords.words('english')
print(stop)
stemmer = SnowballStemmer("english")
def remove_punctuation(test):
    no_punct = "".join([c for c in test if c not in string.punctuation])
    return no_punct
def remove_url(test):
    remove = "".join(c.lower() for c in re.sub(r'[a-z]*[:.]+\S+', " ", test) )
    return remove
def remove_number(test):
    remove = "".join(c if c.isalpha() else ' ' for c in test)
    return remove
def word_stemmer(test):
    stem = "".join([stemmer.stem(i) for i in test] )
    return stem
df['text_split']=df['article'].apply(lambda x:' '.join([word for word in  nltk.RegexpTokenizer(pattern=r'\s+',gaps=True).tokenize(x) if word not in (stop)]))
df['text_url']=df['text_split'].apply(lambda x:remove_url(x))
df['text_clean']=df['text_url'].apply(lambda x:remove_number(x))
df['text_stem']=df['text_clean'].apply(lambda x:word_stemmer(x))

df['text_stem'].head()
#re.sub(r'\W|\b\w*\d\b', " ", test)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

0    what s wrong that  it s history know  mary bea...
1      switzerland  i m guessing   so  outlier indeed 
2      what selling   are  trying to  make murderers  
3      dr  paul powers back immortal fear  a medica...
4     support  prompt browser notifications periodi...
Name: text_stem, dtype: object

In [5]:
import collections
from sklearn.model_selection import train_test_split
counter = collections.Counter()
maxlen = 0
for words in df['text_stem']:
    words = [x.lower() for x in nltk.word_tokenize(words)]
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:    
        counter[word] += 1
print(maxlen)
print(counter)


540
Counter({'i': 6354, 's': 3317, 'the': 2319, 'it': 1790, 'like': 1271, 'you': 1257, 'people': 1208, 't': 1164, 'one': 1072, 'this': 1058, 'trump': 1039, 'they': 961, 'that': 941, 'get': 937, 'would': 918, 'a': 844, 'm': 828, 'we': 730, 'know': 723, 'good': 690, 'us': 679, 'time': 674, 'and': 648, 'he': 647, 'right': 639, 'see': 617, 'if': 616, 'think': 605, 'new': 586, 'white': 586, 'can': 572, 'is': 530, 'what': 527, 'in': 522, 'not': 522, 'want': 520, 'back': 488, 'going': 482, 'even': 482, 'need': 482, 'gab': 481, 'go': 479, 're': 476, 'never': 476, 'there': 473, 'make': 470, 'no': 442, 'to': 437, 'well': 435, 'way': 435, 'so': 423, 'now': 413, 'really': 409, 'all': 396, 'take': 395, 'say': 394, 'still': 392, 'got': 387, 'day': 384, 'great': 382, 'much': 381, 'twitter': 377, 'many': 370, 'world': 369, 'left': 358, 'but': 349, 'free': 346, 'here': 343, 'd': 342, 'man': 342, 'for': 342, 've': 341, 'also': 339, 'god': 334, 'could': 333, 'jews': 324, 'first': 322, 'don': 321, 'love':

In [6]:
from keras.layers.core import Dense, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.layers import Flatten, LSTM
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [9]:
VOCAB_SIZE = 32000
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1
vocab_sz = len(word2index) + 1
index2word = {v:k for k, v in word2index.items()}
print(len(word2index))

28592


In [14]:
from keras.preprocessing import sequence
xs, ys = [], []
for x in df['label']:
    ys.append(x)
for words in df['text_stem']:
    words = [x.lower() for x in nltk.word_tokenize(words)]
    wids = [word2index[word] for word in words]
    xs.append(wids)
Xtrain = sequence.pad_sequences(xs, maxlen=maxlen)
Ytrain = np_utils.to_categorical(ys,num_classes=2)
print(df['label'])
#Y = np_utils.to_categorical(ys,num_classes=2)

0        0
1        0
2        0
3        0
4        0
        ..
19088    0
19089    0
19090    0
19091    0
19092    1
Name: label, Length: 19037, dtype: int64


In [15]:
from keras.preprocessing import sequence
xs, ys = [], []
for x in df1['label']:
    ys.append(x)
for words in df1['article']:
    words = [x.lower() for x in nltk.word_tokenize(words)]
    wids = [word2index[word] for word in words]
    xs.append(wids)
Xtest = sequence.pad_sequences(xs, maxlen=maxlen)
Ytest= np_utils.to_categorical(ys,num_classes=2)
print(df1['label'])
#Y = np_utils.to_categorical(ys,num_classes=2)

0       0
1       1
2       0
3       0
4       1
       ..
4128    0
4129    0
4130    0
4131    0
4132    0
Name: label, Length: 4133, dtype: int64


In [16]:
from keras.preprocessing import sequence
xs, ys = [], []
for x in df2['label']:
    ys.append(x)
for words in df2['article']:
    words = [x.lower() for x in nltk.word_tokenize(words)]
    wids = [word2index[word] for word in words]
    xs.append(wids)
Xdev = sequence.pad_sequences(xs, maxlen=maxlen)
Ydev= np_utils.to_categorical(ys,num_classes=2)
print(df2['label'])
#Y = np_utils.to_categorical(ys,num_classes=2)

0       0
1       0
2       0
3       0
4       0
       ..
4047    0
4048    0
4049    0
4050    0
4051    0
Name: label, Length: 4052, dtype: int64


In [17]:
print(Xtrain.shape, Xtest.shape,Xdev.shape, Ytrain.shape, Ytest.shape,Ydev.shape)

(19037, 540) (4133, 540) (4052, 540) (19037, 2) (4133, 2) (4052, 2)


In [18]:
print(Ytrain)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [19]:
from imblearn.over_sampling import SMOTE
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

smote = SMOTE(random_state=777,k_neighbors=5)
X_smote,y_smote = smote.fit_sample(Xtrain,Ytrain)
y_smote=np_utils.to_categorical(y_smote,num_classes=2)


#print(X_smote,y_smote )
print(X_smote.shape)
print(y_smote.shape)
print(y_smote)


(33686, 540)
(33686, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [20]:
from keras.layers import Flatten, LSTM
sequence_length = X_smote.shape[1] 
vocabulary_size = vocab_sz 
embedding_dim = 1000
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5
epochs = 3
batch_size = 30

In [21]:
# this returns a tensor
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1,maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=2, activation='softmax')(dropout)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

Creating Model...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [22]:
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
print("Traning Model...")
model.fit(X_smote, y_smote, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpoint], validation_data=(Xdev, Ydev))  # starts training

Traning Model...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 33686 samples, validate on 4052 samples
Epoch 1/3
33686/33686 [==============================] - 5148s 153ms/step - loss: 0.6168 - accuracy: 0.6554 - val_loss: 2.2048 - val_accuracy: 0.1597
Epoch 2/3


c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


33686/33686 [==============================] - 5052s 150ms/step - loss: 0.4270 - accuracy: 0.8176 - val_loss: 3.7723 - val_accuracy: 0.1567
Epoch 3/3
33686/33686 [==============================] - 6566s 195ms/step - loss: 0.2787 - accuracy: 0.8998 - val_loss: 5.0158 - val_accuracy: 0.1555


In [23]:
EMBED_SIZE = 400
model = Sequential()
model.add(Embedding(output_dim=EMBED_SIZE, 
                    input_dim=vocab_sz,  
                    input_length=maxlen))
model.add(LSTM(50))
model.add(Dropout(0.25))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation="softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 540, 400)          11437200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                90200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 11,532,702
Trainable params: 11,532,702
Non-trainable params: 0
__________________________________________

In [24]:
BATCH_SIZE =  64
NUM_EPOCHS = 5
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["accuracy"])
history = model.fit(X_smote, y_smote, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS,validation_data=(Xdev, Ydev)) 

Train on 33686 samples, validate on 4052 samples
Epoch 1/5
33686/33686 [==============================] - 596s 18ms/step - loss: 0.6112 - accuracy: 0.6660 - val_loss: 0.5372 - val_accuracy: 0.7475
Epoch 2/5
33686/33686 [==============================] - 591s 18ms/step - loss: 0.3879 - accuracy: 0.8273 - val_loss: 0.7176 - val_accuracy: 0.5891
Epoch 3/5
33686/33686 [==============================] - 590s 18ms/step - loss: 0.1892 - accuracy: 0.9228 - val_loss: 1.0134 - val_accuracy: 0.4894
Epoch 4/5
33686/33686 [==============================] - 592s 18ms/step - loss: 0.0945 - accuracy: 0.9623 - val_loss: 1.1167 - val_accuracy: 0.5612
Epoch 5/5
33686/33686 [==============================] - 591s 18ms/step - loss: 0.0555 - accuracy: 0.9778 - val_loss: 1.3233 - val_accuracy: 0.5671


In [25]:
from sklearn import metrics

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

Ypred = model.predict(Xtest)
for i in range(len(Ypred)):
        max_value=max(Ypred[i])
        for j in range(len(Ypred[i])):
            if max_value==Ypred[i][j]:
                Ypred[i][j]=1
            else:
                Ypred[i][j]=0
score = model.evaluate(Xtest, Ytest, verbose=1)


print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))
print(f1_score(Ytest, Ypred, average="macro"))
print(recall_score(Ytest, Ypred, average="macro"))
print(precision_score(Ytest, Ypred, average="macro"))

print(classification_report(Ytest, Ypred))

4133/4133 [==============================] - 21s 5ms/step
Test score: 1.330, accuracy: 0.568
0.47264692838853184
0.5917867111737791
0.5375785576680201
              precision    recall  f1-score   support

           0       0.92      0.56      0.70      3658
           1       0.16      0.62      0.25       475

   micro avg       0.57      0.57      0.57      4133
   macro avg       0.54      0.59      0.47      4133
weighted avg       0.83      0.57      0.65      4133
 samples avg       0.57      0.57      0.57      4133



In [26]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
X=df['text_stem']
y=df['label']
X1=df1['article']
y1=df1['label']
X2=df2['article']
y2=df2['label']
vect=TfidfVectorizer(analyzer='word',encoding='latin-9')
vect.fit(X)
vect.fit(X1)
vect.fit(X2)
x1=vect.transform(X)
x=x1.toarray()
x2=vect.transform(X1)
x=x2.toarray()
x3=vect.transform(X2)
x=x3.toarray()
chi1 = SelectKBest(chi2, k='all')    
X_train_chi2 = chi1.fit(x1,y)
X_new =X_train_chi2.transform(x1)
X1_train_chi2 = chi1.fit(x2,y1)
X1_new =X1_train_chi2.transform(x2)
X2_train_chi2 = chi1.fit(x3,y2)
X2_new =X2_train_chi2.transform(x3)
print(X_new.shape)
print(X1_new.shape)
print(X2_new.shape)
print(y.shape)


(19037, 14343)
(4133, 14343)
(4052, 14343)
(19037,)


In [27]:
from imblearn.over_sampling import SMOTE
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

smote = SMOTE(random_state=777,k_neighbors=5)
X_smote,y_smote = smote.fit_sample(X_new,y)
#y_smote=np_utils.to_categorical(y_smote,num_classes=2)

vect.inverse_transform(X_smote)
#print(X_smote,y_smote )



[array(['wrong', 'what', 'vagina', 'told', 'that', 'mary', 'know', 'it',
        'history', 'beard'], dtype='<U86'),
 array(['switzerland', 'so', 'indeed', 'guessing'], dtype='<U86'),
 array(['what', 'trying', 'to', 'selling', 'murderers', 'make', 'are'],
       dtype='<U86'),
 array(['thriller', 'powers', 'paul', 'medical', 'killer', 'immortal',
        'hunting', 'fear', 'dr', 'back'], dtype='<U86'),
 array(['thanks', 'support', 'notifications', 'no', 'nice', 'it', 'could',
        'comes', 'choice', 'browser', 'back'], dtype='<U86'),
 array(['well', 'weinstein', 'waiting', 'virtue', 'storm', 'sessions',
        'really', 'patience', 'hope', 'coming', 'beginning'], dtype='<U86'),
 array(['start', 'soon', 'reach', 'populists', 'of', 'new', 'migrants',
        'may', 'mass', 'italy', 'govt', 'form', 'deportation', 'deal',
        'as'], dtype='<U86'),
 array(['mental', 'lord', 'level'], dtype='<U86'),
 array(['votes', 'upon', 'story', 'site', 'runs', 'risk', 'one', 'office',
        'l

In [28]:
print(X_smote.shape)
print(y_smote.shape)
print(y_smote)

(33686, 14343)
(33686,)
0        0
1        0
2        0
3        0
4        0
        ..
33681    1
33682    1
33683    1
33684    1
33685    1
Name: label, Length: 33686, dtype: int64


In [29]:
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
clf =svm.LinearSVC(C=20, penalty='l2')

clf.fit(X_smote,y_smote)
y_pred = clf.predict(X1_new)
#y_pre = clf.predict(X1_new)
#scores = cross_val_score(clf, X1_new, y1, cv=10)
#print("Accuracy: %0.5f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#scores1 = cross_val_score(clf,X1_new, y1, cv=10, scoring='f1_macro')
#print("F1: %0.5f (+/- %0.2f) "% (scores1.mean(), scores.std() * 2))
#scores2 = cross_val_score(clf,X1_new, y1, cv=10, scoring='recall_macro')
#print("recall: %0.5f (+/- %0.2f) "% (scores2.mean(), scores.std() * 2))
#scores3 = cross_val_score(clf,X1_new, y1, cv=10, scoring='precision_macro')
#print("precision: %0.5f (+/- %0.2f)" % (scores3.mean(), scores3.std() * 2))
print(f1_score(y1, y_pred, average="macro"))
print(recall_score(y1, y_pred, average="macro"))
print(precision_score(y1, y_pred, average="macro"))
print(classification_report(y1, y_pred))

0.5713390787748713
0.5948709389657851
0.5643700217035956
              precision    recall  f1-score   support

           0       0.91      0.84      0.87      3658
           1       0.22      0.35      0.27       475

    accuracy                           0.78      4133
   macro avg       0.56      0.59      0.57      4133
weighted avg       0.83      0.78      0.80      4133



c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [30]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(C=10,penalty='l2')
lr.fit(X_smote,y_smote)
y_pred1 = lr.predict(X1_new)

print(f1_score(y1, y_pred1, average="macro"))
print(recall_score(y1, y_pred1, average="macro"))
print(precision_score(y1, y_pred1, average="macro"))
print(classification_report(y1, y_pred1))

0.6034065011394986
0.621157089004633
0.5935728699950217
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      3658
           1       0.27      0.37      0.31       475

    accuracy                           0.81      4133
   macro avg       0.59      0.62      0.60      4133
weighted avg       0.84      0.81      0.83      4133



c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
